In [1]:
import os
from os.path import isfile
import pandas as pd
import pickle
from compute_ale import compute_ale
from compile_studies import compile_studies
from contribution import contribution
%matplotlib inline

cwd = os.getcwd()
raw_folder = cwd + '/DataRaw/'
pickle_folder = cwd + '/DataPickle/'

'''filename = askopenfilename()
df = pd.read_excel(filename, engine='openpyxl', header=None)'''

df = pd.read_excel(raw_folder + 'Metastocalculate.xlsx', engine='openpyxl', header=None)
with open(pickle_folder + 'experiments.pickle', 'rb') as f:
    experiments, tasks = pickle.load(f)

In [2]:
for row in df.itertuples():
    if row[1] == 'M': #Main Effect Analysis
        s_index = compile_studies(df, row[0], experiments, tasks)
        if len(s_index) >= 12: 
            print('{} : {} experiments; average of {:.2f} subjects per experiment'.format(row[2], len(s_index), experiments.iloc[s_index].Subjects.mean()))
            
            
        else:
            print("{} : only {} experiments - not analyzed!".format(category, len(s_index)))

Other : 19 experiments; average of 22.26 subjects per experiment
Cwclassic : 37 experiments; average of 25.76 subjects per experiment
Cwvariant : 21 experiments; average of 26.33 subjects per experiment
CwIvsC : 58 experiments; average of 25.97 subjects per experiment
CwIvsN : 41 experiments; average of 22.22 subjects per experiment
CwIvsNwords : 17 experiments; average of 22.24 subjects per experiment
CwIvsNsymbolsletters : 23 experiments; average of 22.30 subjects per experiment


In [3]:
s_index = compile_studies(df, 0, experiments, tasks)
experiments_s = experiments.loc[s_index].reset_index(drop=True)
study = 'Other'


compute_ale(s_index, experiments_s, study, noise_repeat=5000)
contribution(s_index, experiments_s, tasks, study)

Other - loading Foci
Other - loading ALE
Other - loading null PDF
Other - loading p-values
Other - loading noise
Other - inference and printing
Min p-value for FWE:0.043
Min p-value for cFWE:0.0
Min p-value for TFCE:0.0
Other - done!
entered except


#### Contrast

In [4]:
def par_perm(s1,s2,vp,a,benchmark):
    sr = np.arange(len(s1)+len(s2))
    np.random.shuffle(sr)
    draw = (1-np.prod(vp[sr[:len(s1)],:], axis=0)) - (1-np.prod(vp[sr[len(s1):],:], axis=0))
    a[draw > benchmark] += draw[draw > benchmark]
    return a

In [5]:
from os import getcwd
from os.path import isfile
import nibabel as nb
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
from scipy.stats import norm

In [6]:
s1 = compile_studies(df, 3, experiments, tasks)
experiments1 = experiments.loc[s1].reset_index(drop=True)
study1 = df.loc[3,1]
s1 = list(range(len(s1)))
s2 = compile_studies(df, 4, experiments, tasks)
experiments2 = experiments.loc[s2].reset_index(drop=True)
study2 = df.loc[4,1]
s2 = list(range(len(s2)))

In [7]:
if len(s1) >= 12 and len(s2) >= 12:
    if not isfile("{}/ALE/Results/{}_cFWE05.nii".format(cwd, study1)):
        compute_ale(s1, experiments1, study1, noise_repeat=5000)
    if not isfile("{}/ALE/Results/{}_cFWE05.nii".format(cwd, study2)):
        compute_ale(s2, experiments2, study2, noise_repeat=5000)

Cwclassic - loading Foci
Cwclassic - loading ALE
Cwclassic - loading null PDF
Cwclassic - loading p-values
Cwclassic - loading noise
Cwclassic - inference and printing
Min p-value for FWE:0.0
Min p-value for cFWE:0.0
Min p-value for TFCE:0.0
Cwclassic - done!


In [8]:
u = 0.05

In [10]:
cwd = getcwd()
mask_folder = cwd + "/MaskenEtc/"

try:
    os.makedirs(cwd + "/ALE/Contrasts/Images")
    os.makedirs(cwd + "/ALE/Conjunctions/Images")
except:
    pass

template = nb.load(mask_folder + "Grey10.nii")
template_data = template.get_fdata()
template_shape = template_data.shape
pad_tmp_shape = [value+30 for value in template_shape]

repeats=25000
corr = ["cFWE", "TFCE"]

if not isfile("{}/ALE/Contrasts/{}--{}_P95.nii".format(cwd, study1, study2)):
    print("Positive Contrast.")
    fx = nb.load("{}/ALE/Results/{}_{}05.nii".format(cwd,study1,corr[0])).get_fdata()
    ind = np.where(fx > 0)
    if ind[0].size > 0:
        z1 = fx[fx > 0]
        vp = np.zeros((len(s1)+len(s2), z1.size))
        a = np.zeros(ind[0].shape)
        
        for i in s1:
            data = np.zeros(pad_tmp_shape)
            for ii in range(experiments1.at[i, 'Peaks']):
                coords = experiments1.XYZ[i].T[:,:3][ii]
                x_range = (coords[0],coords[0]+31)
                y_range = (coords[1],coords[1]+31)
                z_range = (coords[2],coords[2]+31)
                data[x_range[0]:x_range[1], y_range[0]:y_range[1], z_range[0]:z_range[1]] = \
                np.maximum(data[x_range[0]:x_range[1], y_range[0]:y_range[1], z_range[0]:z_range[1]],
                           experiments1.at[i, 'Kernel'])
            data = data[15:data.shape[0]-15,15:data.shape[1]-15, 15:data.shape[2]-15]
            vp[i,:] = data[tuple(ind)]
        
        for i in s2:
            data = np.zeros(pad_tmp_shape)
            for ii in range(experiments2.at[i, 'Peaks']):
                coords = experiments2.XYZ[i].T[:,:3][ii]
                x_range = (coords[0],coords[0]+31)
                y_range = (coords[1],coords[1]+31)
                z_range = (coords[2],coords[2]+31)
                data[x_range[0]:x_range[1], y_range[0]:y_range[1], z_range[0]:z_range[1]] = \
                np.maximum(data[x_range[0]:x_range[1], y_range[0]:y_range[1], z_range[0]:z_range[1]],
                           experiments2.at[i, 'Kernel'])
            data = data[15:data.shape[0]-15,15:data.shape[1]-15, 15:data.shape[2]-15]
            vp[i+len(s1),:] = data[tuple(ind)]
        
        vp = 1-vp
        benchmark = (1-np.prod(vp[:len(s1),:], axis=0)) - (1-np.prod(vp[len(s1):,:], axis=0))
        
        print("Randomising.")
        a = Parallel(n_jobs=-1,backend="threading", verbose=1)(delayed(par_perm)(s1, s2, vp, a, benchmark) for i in range(repeats))
        
    else:
        print("{}: No significant indices!".format(study1))
            




Positive Contrast.
Randomising.


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 11234 tasks  

In [23]:
a = np.sum(a, axis=0)

/usr/local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [13]:
a = norm.ppf(1-(a/repeats))

In [15]:
a = np.nan_to_num(a)

In [19]:
tfce1 = nb.load(cwd + "/ALE/Results/Other_TFCE05.nii").get_fdata()

In [22]:
tfce1[tfce1 > 0]

array([39252.15963478, 40319.35824362, 41230.07331941, 40808.05875177,
       40497.06338385, 41230.07331941, 40934.99919305, 39711.92408612,
       39252.15963478, 41197.44013787, 40108.97230703, 40108.97230703,
       38945.00099189, 40126.43212874, 40934.99919305, 40497.06338385,
       40656.7296134 , 41230.07331941, 40808.05875177, 39493.2979982 ,
       39252.15963478, 40126.43212874, 39493.2979982 , 38945.00099189,
       38190.62966111, 40108.97230703, 41714.62946921, 40662.68773335,
       40108.97230703, 44279.486089  , 43890.22340534, 41714.62946921,
       38190.62966111, 38945.00099189, 40662.68773335, 41714.62946921,
       41940.5398059 , 41854.24736713, 43475.26983895, 46899.95234882,
       46685.10547034, 44638.80058799, 41197.44013787, 38945.00099189,
       38945.00099189, 39531.56770909, 39531.56770909, 38190.62966111,
       42612.81848709, 46445.09319548, 46170.65761705, 43890.22340534,
       38945.00099189, 42168.84739905, 42168.84739905, 39252.15963478,
      

In [18]:
a[a > 0].shape

(171,)